# CÓDIGO CREAR GALERIA DE IMAGENES PRESENTARLAS EN HTML

In [3]:
import pandas as pd
import os
from pathlib import Path

# 1. CREAR DATAFRAME CON LAS IMÁGENES
def crear_dataframe_imagenes(carpeta_imagenes='imagenes_tenis'):
    """Crea DataFrame con nombres y rutas de todas las imágenes"""
    carpeta = Path(carpeta_imagenes)
    
    if not carpeta.exists():
        print(f"❌ La carpeta '{carpeta_imagenes}' no existe")
        return None
    
    # Buscar todos los archivos de imagen
    extensiones = ['*.jpg', '*.JPG', '*.jpeg', '*.JPEG']
    archivos = []
    for extension in extensiones:
        archivos.extend(carpeta.glob(extension))
    
    # Crear DataFrame
    datos = [{'nombre_archivo': archivo.name, 'ruta': str(archivo)} for archivo in archivos]
    df = pd.DataFrame(datos)
    
    print(f"✅ DataFrame creado con {len(df)} imágenes")
    return df

# 2. GENERAR HTML PAGINADO
def generar_html_paginado(df, altura_cm=5, ancho_cm=8, imagenes_por_pagina=100):
    """Genera múltiples archivos HTML con paginación para todas las imágenes"""
    
    altura_px = int(altura_cm * 37.8)
    ancho_px = int(ancho_cm * 37.8)
    total_imagenes = len(df)
    total_paginas = (total_imagenes + imagenes_por_pagina - 1) // imagenes_por_pagina
    
    print(f"📚 Generando {total_paginas} páginas con {imagenes_por_pagina} imágenes cada una...")
    
    for pagina in range(total_paginas):
        inicio = pagina * imagenes_por_pagina
        fin = min(inicio + imagenes_por_pagina, total_imagenes)
        df_pagina = df.iloc[inicio:fin]
        
        # Contadores para esta página
        imagenes_ok = 0
        imagenes_error = 0
        
        # Generar HTML para la página actual
        html = f"""<!DOCTYPE html>
<html lang="es">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Galería de Imágenes - Página {pagina + 1} de {total_paginas}</title>
    <style>
        body {{
            font-family: Arial, sans-serif;
            margin: 0;
            padding: 20px;
            background-color: #f5f5f5;
        }}
        .container {{
            max-width: 1400px;
            margin: 0 auto;
            background: white;
            padding: 20px;
            border-radius: 10px;
            box-shadow: 0 2px 10px rgba(0,0,0,0.1);
        }}
        .header {{
            text-align: center;
            margin-bottom: 20px;
            padding-bottom: 15px;
            border-bottom: 2px solid #e0e0e0;
        }}
        .header h1 {{
            color: #2c3e50;
            margin: 0 0 10px 0;
        }}
        .paginacion {{
            text-align: center;
            margin: 20px 0;
        }}
        .paginacion a {{
            display: inline-block;
            padding: 8px 16px;
            margin: 0 5px;
            background: #3498db;
            color: white;
            text-decoration: none;
            border-radius: 5px;
            font-size: 14px;
        }}
        .paginacion a:hover {{
            background: #2980b9;
        }}
        .paginacion .pagina-actual {{
            background: #2c3e50;
        }}
        .tabla-imagenes {{
            width: 100%;
            border-collapse: collapse;
        }}
        .tabla-imagenes th {{
            background-color: #34495e;
            color: white;
            padding: 12px;
            text-align: left;
            font-size: 14px;
            position: sticky;
            top: 0;
        }}
        .tabla-imagenes td {{
            padding: 10px;
            border-bottom: 1px solid #ecf0f1;
            vertical-align: middle;
        }}
        .tabla-imagenes tr:nth-child(even) {{
            background-color: #f8f9fa;
        }}
        .tabla-imagenes tr:hover {{
            background-color: #e8f4f8;
        }}
        .nombre-archivo {{
            font-family: 'Courier New', monospace;
            font-size: 11px;
            font-weight: bold;
            word-break: break-all;
            max-width: 250px;
        }}
        .imagen-cell {{
            text-align: center;
        }}
        .imagen-container {{
            display: inline-block;
            padding: 3px;
            background: white;
            border: 1px solid #bdc3c7;
            border-radius: 3px;
        }}
        img {{
            display: block;
            background-color: #f9f9f9;
        }}
        .error {{
            color: #e74c3c;
            font-weight: bold;
            font-style: italic;
        }}
        .footer {{
            text-align: center;
            margin-top: 25px;
            padding-top: 15px;
            border-top: 1px solid #ecf0f1;
            color: #7f8c8d;
            font-size: 12px;
        }}
    </style>
</head>
<body>
    <div class="container">
        <div class="header">
            <h1>🖼️ Galería de Imágenes</h1>
            <div>Página {pagina + 1} de {total_paginas} | Imágenes {inicio + 1} a {fin} de {total_imagenes}</div>
        </div>
        
        <div class="paginacion">
            {"".join([f'<a href="galeria_pagina_{i+1}.html" class="{"pagina-actual" if i == pagina else ""}">{i+1}</a>' 
                     for i in range(total_paginas)])}
        </div>
        
        <table class="tabla-imagenes">
            <tr>
                <th width="25%">Nombre del Archivo</th>
                <th width="75%">Imagen ({altura_cm} × {ancho_cm} cm)</th>
            </tr>
        """
        
        # Agregar filas con imágenes
        for _, fila in df_pagina.iterrows():
            if os.path.exists(fila['ruta']):
                html += f"""
                <tr>
                    <td class="nombre-archivo">{fila['nombre_archivo']}</td>
                    <td class="imagen-cell">
                        <div class="imagen-container">
                            <img src="{fila['ruta']}" width="{ancho_px}" height="{altura_px}" 
                                 style="object-fit: contain;"
                                 alt="{fila['nombre_archivo']}">
                        </div>
                    </td>
                </tr>
                """
                imagenes_ok += 1
            else:
                html += f"""
                <tr>
                    <td class="nombre-archivo">{fila['nombre_archivo']}</td>
                    <td class="error">❌ Archivo no encontrado</td>
                </tr>
                """
                imagenes_error += 1
        
        html += f"""
        </table>
        
        <div class="paginacion">
            {"".join([f'<a href="galeria_pagina_{i+1}.html" class="{"pagina-actual" if i == pagina else ""}">{i+1}</a>' 
                     for i in range(total_paginas)])}
        </div>
        
        <div class="footer">
            <p>✅ Imágenes mostradas: {imagenes_ok} | ❌ Errores: {imagenes_error} | 📅 Generado: {pd.Timestamp.now().strftime('%Y-%m-%d %H:%M')}</p>
        </div>
    </div>
</body>
</html>"""
        
        # Guardar archivo HTML
        archivo_salida = f'galeria_pagina_{pagina + 1}.html'
        with open(archivo_salida, 'w', encoding='utf-8') as f:
            f.write(html)
        
        print(f"✅ Página {pagina + 1}/{total_paginas}: {archivo_salida} - {imagenes_ok} imágenes OK")
    
    print(f"\n🎉 GENERACIÓN COMPLETADA")
    print(f"📊 Total de páginas: {total_paginas}")
    print(f"📸 Total de imágenes: {total_imagenes}")
    print(f"📁 Archivos generados: galeria_pagina_1.html a galeria_pagina_{total_paginas}.html")
    print(f"🌐 Abre 'galeria_pagina_1.html' en tu navegador para comenzar")

# 3. EJECUCIÓN PRINCIPAL
print("🚀 INICIANDO GENERACIÓN DE GALERÍA HTML")
print("=" * 50)

# Crear DataFrame
df_imagenes = crear_dataframe_imagenes('imagenes_tenis')

if df_imagenes is not None:
    # Generar HTML paginado
    generar_html_paginado(
        df=df_imagenes,
        altura_cm=5,
        ancho_cm=5,
        imagenes_por_pagina=100  # 100 imágenes por página = 26 páginas para 2560 fotos
    )
else:
    print("❌ No se pudo crear el DataFrame. Verifica la carpeta 'imagenes_tenis'")

🚀 INICIANDO GENERACIÓN DE GALERÍA HTML
✅ DataFrame creado con 564 imágenes
📚 Generando 6 páginas con 100 imágenes cada una...
✅ Página 1/6: galeria_pagina_1.html - 100 imágenes OK
✅ Página 2/6: galeria_pagina_2.html - 100 imágenes OK
✅ Página 3/6: galeria_pagina_3.html - 100 imágenes OK
✅ Página 4/6: galeria_pagina_4.html - 100 imágenes OK
✅ Página 5/6: galeria_pagina_5.html - 100 imágenes OK
✅ Página 6/6: galeria_pagina_6.html - 64 imágenes OK

🎉 GENERACIÓN COMPLETADA
📊 Total de páginas: 6
📸 Total de imágenes: 564
📁 Archivos generados: galeria_pagina_1.html a galeria_pagina_6.html
🌐 Abre 'galeria_pagina_1.html' en tu navegador para comenzar


In [4]:
import os

# --- Configuración ---
carpeta_imagenes = "imagenes_tenis"   # cambia si es necesario
archivo_salida = "galeria_tenis.html"
ancho_cm = 7                          # tamaño deseado
ancho_px = int(ancho_cm * 37.79527559)  # conversión cm → px (1 cm ≈ 37.8 px)

# --- Obtener lista de imágenes ---
extensiones_validas = (".jpg", ".jpeg", ".png", ".bmp", ".gif", ".webp")
imagenes = sorted([f for f in os.listdir(carpeta_imagenes)
                   if f.lower().endswith(extensiones_validas)])

# --- Crear HTML ---
html = f"""<!DOCTYPE html>
<html lang="es">
<head>
<meta charset="UTF-8">
<title>Galería de Imágenes de Tenis</title>
<style>
  body {{
    font-family: Arial, sans-serif;
    background: #f8f8f8;
    color: #222;
  }}
  table {{
    border-collapse: collapse;
    width: 100%;
  }}
  td {{
    border-bottom: 1px solid #ccc;
    padding: 8px;
    vertical-align: middle;
  }}
  .nombre {{
    font-size: 14pt;
    width: 40%;
    word-wrap: break-word;
  }}
  .imagen img {{
    width: {ancho_px}px;
    height: auto;
    border-radius: 6px;
  }}
</style>
</head>
<body>
<h2>Galería de Imágenes ({len(imagenes)} archivos)</h2>
<table>
"""

for nombre in imagenes:
    ruta = os.path.join(carpeta_imagenes, nombre).replace("\\", "/")
    html += f"""
    <tr>
      <td class="nombre">{nombre}</td>
      <td class="imagen"><img src="{ruta}" alt="{nombre}"></td>
    </tr>
    """

html += """
</table>
</body>
</html>
"""

# --- Guardar archivo ---
with open(archivo_salida, "w", encoding="utf-8") as f:
    f.write(html)

print(f"✅ Galería creada: {archivo_salida}")


✅ Galería creada: galeria_tenis.html
